In [16]:
import pandas as pd
import numpy as np
import os
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import Callback
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras.metrics import mean_absolute_percentage_error

In [2]:
from data_reader import z_score_inv

In [3]:
from next_batch import LSTM_WINDOW_SIZE, INPUT_SIZE, PREDICTORS
from next_batch import get_trainable_data

In [4]:
import os
os.environ['HOME']

'/Users/lijinyang'

In [5]:
plt.ion()

DATA_FILE = 'data.npz'
if not os.path.exists(DATA_FILE):
    (x_train, y_train), (x_test, y_test), mean, std = get_trainable_data()
    np.savez_compressed('data.npz', x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test,
                        mean=mean, std=std)
else:
    d = np.load(DATA_FILE)
    x_train = d['x_train']
    y_train = d['y_train']
    x_test = d['x_test']
    y_test = d['y_test']
    mean = d['mean']
    std = d['std']

print('x_train.shape =', x_train.shape)
print('y_train.shape =', y_train.shape)

print('x_test.shape =', x_test.shape)
print('y_test.shape =', y_test.shape)

x_train.shape = (2122, 10, 29)
y_train.shape = (2122, 1)
x_test.shape = (616, 10, 29)
y_test.shape = (616, 1)


In [6]:
PREDICTORS = PREDICTORS[:5]

In [ ]:
PREDICTORS

In [ ]:
def print_np_arr(x):
    return np.array_repr(x).replace('\n', '')


def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


class Monitor(Callback):

    def __init__(self, inputs):
        self.inputs = inputs

    def on_epoch_end(self, epoch, logs=None):
        np.set_printoptions(precision=6, suppress=True)

        # print('\n\n')
        # print('_' * 80)

        # TODO: make it with pandas and better.
        predictions = self.model.predict(self.inputs)
        # TODO: should be the mean_sigma of std_sigma only
        pred_sigmas = [z_score_inv(pred, mean, std) for pred in predictions.flatten()]
        true_sigmas = [z_score_inv(true, mean, std) for true in y_test.flatten()]
        dummy_sigmas = [z_score_inv(dummy, mean, std) for dummy in np.roll(y_test.flatten(), shift=1)]

        #if 'DISPLAY' not in os.environ:
        #    plt.clf()
        #    plt.plot(true_sigmas, color='blue')
        #    plt.plot(pred_sigmas, color='lime')
        #    plt.pause(0.001)
        #    plt.show()

        test_mape = mean_absolute_percentage_error(np.array(true_sigmas), np.array(pred_sigmas))
        dummy_mape = mean_absolute_percentage_error(np.array(true_sigmas), np.array(dummy_sigmas))
        print('[{0}] test = {1:.3f}, test_dummy = {2:.3f}, '
              'train = {3:.3f}, val = {4:.3f}.'.format(str(epoch).zfill(4), test_mape, dummy_mape,
                                                       logs['loss'], logs['val_loss']))
        
        logs['MAPE']=test_mape
        # num_values_to_predict = 10
        # r_train_idx = randint(a=0, b=len(x_train) - num_values_to_predict)
        # print('pred train  =',
        #       print_np_arr(self.model.predict(x_train[r_train_idx:r_train_idx + num_values_to_predict]).flatten()))
        # print('truth train =', print_np_arr(y_train[r_train_idx:r_train_idx + num_values_to_predict].flatten()))
        # r_test_idx = randint(a=0, b=len(x_test) - num_values_to_predict)
        # print('pred  test  =',
        #       print_np_arr(self.model.predict(x_test[r_test_idx:r_test_idx + num_values_to_predict]).flatten()))
        # print('truth test  =', print_np_arr(y_test[r_test_idx:r_test_idx + num_values_to_predict].flatten()))
        # print('_' * 80)
        # print('\n')


m = Sequential()
m.add(LSTM(32, input_shape=(LSTM_WINDOW_SIZE, INPUT_SIZE)))
# m.add(Dropout(0.1))
m.add(Dense(16, activation='sigmoid'))
# m.add(Dropout(0.1))
m.add(Dense(1, activation='linear'))


# PAPER: with mean absolute percent error (MAPE) as the objective loss function
# PAPER: The model is trained by the 'Adam' method

def sigma_loss(y_true, y_pred):
    real_y_true = y_true * std + mean
    real_y_pred = y_pred * std + mean
    return K.mean(K.abs(real_y_true - real_y_pred) / real_y_true) * 100


m.compile(optimizer=Adam(lr=0.0001), loss=sigma_loss)  # mape
m.summary()

for until_predictor_id in range(4, len(PREDICTORS)):

    try:
        print('Now we have {}/{} predictors.'.format(until_predictor_id + 1, len(PREDICTORS)))
        mask_train = np.zeros_like(x_train)
        mask_test = np.zeros_like(x_test)

        mask_train[:, :, 0:until_predictor_id + 1] = 1.0
        mask_test[:, :, 0:until_predictor_id + 1] = 1.0

        x_train_masked = x_train * mask_train
        x_test_masked = x_test * mask_test

        # PAPER: with 32 examples in a batch
        # PAPER:  This can be achieved after roughly 600 epochs.
        monitor = Monitor(inputs=x_test_masked)
        model = m.fit(x_train_masked, y_train,
                      validation_split=0.2,
                      shuffle=True,
                      batch_size=32,
                      epochs=600,
                      verbose=0,
                      callbacks=[monitor])

        # print('Learning rate was {}'.format(K.get_value(m.optimizer.lr)))
        # K.set_value(m.optimizer.lr, K.get_value(m.optimizer.lr) * 0.5)
        # print('Learning rate is now {}'.format(K.get_value(m.optimizer.lr)))

    except KeyboardInterrupt:
        print('Received KeyboardInterrupt. Going to add the next predictor.')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 32)                7936      
_________________________________________________________________
dense_35 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 17        
Total params: 8,481.0
Trainable params: 8,481
Non-trainable params: 0.0
_________________________________________________________________
Now we have 5/5 predictors.
[0000] test = 711.273, test_dummy = 35.730, train = 310.557, val = 546.639.
[0001] test = 646.610, test_dummy = 35.730, train = 258.005, val = 469.668.
[0002] test = 590.251, test_dummy = 35.730, train = 199.106, val = 393.443.
[0003] test = 503.471, test_dummy = 35.730, train = 146.954, val = 270.297.
[0004] test = 397.630, test_dummy = 35.730, train = 112.457, val = 1

[0102] test = 62.112, test_dummy = 35.730, train = 36.337, val = 51.475.
[0103] test = 64.621, test_dummy = 35.730, train = 36.176, val = 56.664.
[0104] test = 64.431, test_dummy = 35.730, train = 36.064, val = 55.399.
[0105] test = 63.287, test_dummy = 35.730, train = 36.036, val = 53.600.
[0106] test = 65.505, test_dummy = 35.730, train = 36.484, val = 57.172.
[0107] test = 65.361, test_dummy = 35.730, train = 36.035, val = 56.789.
[0108] test = 63.035, test_dummy = 35.730, train = 36.076, val = 51.297.
[0109] test = 64.283, test_dummy = 35.730, train = 35.956, val = 53.672.
[0110] test = 65.874, test_dummy = 35.730, train = 35.961, val = 56.334.
[0111] test = 68.039, test_dummy = 35.730, train = 35.906, val = 61.083.
[0112] test = 63.490, test_dummy = 35.730, train = 35.928, val = 52.765.
[0113] test = 68.490, test_dummy = 35.730, train = 35.927, val = 63.101.
[0114] test = 62.823, test_dummy = 35.730, train = 35.750, val = 51.523.
[0115] test = 65.480, test_dummy = 35.730, train = 

[0215] test = 75.395, test_dummy = 35.730, train = 33.730, val = 61.648.
[0216] test = 78.721, test_dummy = 35.730, train = 33.574, val = 51.551.
[0217] test = 79.242, test_dummy = 35.730, train = 33.583, val = 51.083.
[0218] test = 77.174, test_dummy = 35.730, train = 33.830, val = 59.997.
[0219] test = 79.961, test_dummy = 35.730, train = 33.603, val = 52.543.
[0220] test = 77.467, test_dummy = 35.730, train = 33.406, val = 60.129.
[0221] test = 77.498, test_dummy = 35.730, train = 35.135, val = 56.675.
[0222] test = 76.095, test_dummy = 35.730, train = 33.659, val = 57.955.
[0223] test = 77.408, test_dummy = 35.730, train = 33.418, val = 54.576.
[0224] test = 75.276, test_dummy = 35.730, train = 33.400, val = 63.717.
[0225] test = 74.248, test_dummy = 35.730, train = 33.655, val = 58.670.
[0226] test = 75.797, test_dummy = 35.730, train = 33.526, val = 53.613.
[0227] test = 76.716, test_dummy = 35.730, train = 33.542, val = 55.328.
[0228] test = 75.156, test_dummy = 35.730, train = 

[0328] test = 86.688, test_dummy = 35.730, train = 31.898, val = 53.465.
[0329] test = 86.799, test_dummy = 35.730, train = 31.687, val = 53.144.
[0330] test = 84.671, test_dummy = 35.730, train = 31.755, val = 58.767.
[0331] test = 85.604, test_dummy = 35.730, train = 32.161, val = 53.762.
[0332] test = 85.159, test_dummy = 35.730, train = 31.841, val = 54.094.
[0333] test = 81.007, test_dummy = 35.730, train = 31.882, val = 62.281.
[0334] test = 83.542, test_dummy = 35.730, train = 31.628, val = 55.044.
[0335] test = 84.862, test_dummy = 35.730, train = 31.638, val = 57.124.
[0336] test = 85.954, test_dummy = 35.730, train = 31.841, val = 55.984.
[0337] test = 86.441, test_dummy = 35.730, train = 31.621, val = 56.041.
[0338] test = 87.745, test_dummy = 35.730, train = 31.540, val = 54.692.
[0339] test = 85.301, test_dummy = 35.730, train = 31.587, val = 58.786.
[0340] test = 84.430, test_dummy = 35.730, train = 31.642, val = 60.180.
[0341] test = 84.009, test_dummy = 35.730, train = 

In [37]:
model.history['MAPE']

[196.24991780135989,
 143.80184409258163,
 118.17808247199109,
 116.6846411699687,
 128.4595578234875,
 130.99501863824983,
 136.59306687575813,
 142.89380309313754,
 142.888598428424,
 145.11534748495225]